In [ ]:
# calculate top 5 customers with their revenue

CREATE OR REFRESH MATERIALIZED VIEW (
    customer_unique_id string,
    customer_city string,
    customer_state string,
    total_Revenue float,
    CONSTRAINT valid_revenue EXPECT(total_Revenue is not null and total_Revenue > 0) ON VIOLATION DROP ROW
)
COMMENT "Aggregated data to find out top 5 customers based on thier revenue"
TBLPROPERTIES ("quality" = "gold")
with calc_rev as (
select  cust.customer_unique_id,customer_city, customer_state,
        sum(price+freight_value) as total_Revenue
from live.DimCustomers as cust
inner join live.FactOrders as ord
    on cust.customer_id = ord.customer_id
inner join live.FactOrderItems ord_itm
    on ord_itm.order_id = ord.order_id
group by cust.customer_unique_id,customer_city, customer_state
),
rank_cust as (
select *,dense_rank() over(partition by customer_unique_id order by total_Revenue desc ) as rn 
from calc_rev 
)
select * from rank_cust where rn <6